En este archivo puedes escribir lo que estimes conveniente. Te recomendamos detallar tu solución y todas las suposiciones que estás considerando. Aquí puedes ejecutar las funciones que definiste en los otros archivos de la carpeta src, medir el tiempo, memoria, etc.

# DATA ENGINEER CHALLENGE



## CONSIDERACIONES GENERALES

Para el desarrollo de este challenge estoy utilizando:
1. Python 3.11.7
2. Ubuntu 22.04
3. Memoria ram 12 gigas
4. Sin GPU

Estos serán las limitantes del trabajo desarrollado. No se decidió utilizar recursos en la nube porque el archivo analizado no era tan grande y se procedió a que las mejoras de optimización solo sean de código mas no de hardware.

La data que se extrae no necesita un proceso de limpieza y la data no llegará a ser de gran tamaño como para ser considerada BigData.

## Optimizaciones de Memoria

Para la solución de las preguntas de optimización de memoria se ha considerado lo siguiente:

1. Se busca optimizar la memoria más no mantenerla al minimo.
2. El tiempo de ejecución no debería ser excesivo aunque se este buscando optimizar la memoria.
3. El archivo es un ejemplo pero pueden haber casos dónde sean muy extensos.

Como solución se ha considerado utilizar la librería Pandas debido a su fácil manejo y que nos provee agregar argumentos a la función de lectura del archivo JSON para poder optimizar la memoría. Por otro lado, se manejó la alternativa de utilizar una lectura linea a linea del archivo; sin embargo, esto tomaría demasiado tiempo de ejecución en relación al beneficio de ahorrar memoría.

Consideraciones claves para reducir el consumo de memoría con Pandas:

1. Chunksize: Nos permite dividir archivos grandes en pequeños trozos lo cual permite el ahorro de memoría.
2. dtype: Nos permite definir los tipos de datos de cada columna, así se puede asignar tipos de datos menores a los de por defecto. En este caso, tambien se uso para convertir información no necesaria en un tipo de dato booleano ( dado que cuesta mucho menos).
3. Elegir las columnas necesarias para el proceso.
4. Utilizar funciones map y reduce para reducir el uso de memoría y trabajar con los chunks.

## Optimizaciones de tiempo

Para la solución de las preguntas de optimizacion de tiempo se ha considerado lo siguiente:

1. Se busca optimizar el tiempo de ejecución sin dejar de lado el consumo de memoria
2. Limites de hardware propio.
3. La data suministrada no será muy extensa para considerarse BigData.

Como solución para las preguntas de optimización de tiempo se ha considerado entre dos alternativas la utilización de la libreria Polars y la lectura de cada linea utilizando la libreria JSON. Sabemos que para optimizar el tiempo de ejecución es necesario tambien tener en cuenta la memoria consumida, además de la escritura de código eficiente. Por otro lado, la paralelización de las tareas ayuda a acelerar el proceso aunque en Python es complicado debido al GIL existente. Asimismo, el lenguaje interpretado como tal es más lento que lenguajes compiladas como Rust o C++, por tal motivo, para acelerar la ejecución se han buscado opciones de librerias que tengan una API para python. Por ultimo, se ha tenido en cuenta que las operaciones vectoriales también aumentan la velocidad de ejecución.

Por todo lo anterior descrito, Polars es la mejor alternativa para optimizar la lectura de los dataframes (según las consideraciones descritas más arriba).

Consideraciones de [Polars](https://docs.pola.rs/):

1. Escrito en Rust.
2. Trabajo en paralelo.
3. Motor de Consultas Vectorizadas.

Consideraciones del código:

1. Se ha escrito el codigo utilizando lazy API puesto que permite trabajar con datasets grandes y en streaming.

### Mejoras

1. Mejorar el conocimiento de la libreria Polars para saber si existen mejores formas de extraer la data.
2. Pruebas con [Dask](https://docs.dask.org/en/stable/) para aprovechar que puede ser utilizado en sistemas distribuidos.
3. Reescribir algunas funcionalidades de librerias utilizadas, y ver si se aplica memoization.
4. Aplicar multihilo o multiproceso a chunks de pandas

## Descomprimir el archivo

In [1]:
import os
from utils import unzip_json
file_path = "datasets/tweets.json.zip"
output_dir = "datasets/"
output_file_path = "datasets/farmers-protest-tweets-2021-2-4.json"
if not os.path.exists(output_file_path):
    unzip_json(file_path,output_dir)

## Preguntas


### Q1 
Las top 10 fechas donde hay más tweets. Mencionar el usuario (username) que más publicaciones tiene por cada uno de esos días. 

#### Q1 MEMORY

Se considera que un chunksize de 1000 es un valor que no eleva mucho la memoría ni retraza mucho el tiempo de ejecución.

In [3]:
from q1_memory import q1_memory
result = q1_memory(output_file_path)

Filename: /home/admintdp/Documentos/personalProyects/challenge_DE/src/q1_memory.py

Line #    Mem usage    Increment  Occurrences   Line Contents
     8 137.3125 MiB 137.3125 MiB           1   @profile(precision=4)
     9                                         def q1_memory(file_path: str) -> List[Tuple[datetime.date, str]]:
    10                                             # The selected function is the one that has the best performance,
    11                                             # It is the one that uses pandas with the best memory optimization.
    12                                             # The line by line function is the one that uses the least memory but it is the slowest
    13                                             # And for that reason it is not the best option.
    14                                             
    15                                             # OPTIMIZATION:
    16                                             # Many options to optimize 

Resultados:

In [4]:
result

[(datetime.date(2021, 2, 12), 'RanbirS00614606'),
 (datetime.date(2021, 2, 13), 'MaanDee08215437'),
 (datetime.date(2021, 2, 17), 'RaaJVinderkaur'),
 (datetime.date(2021, 2, 16), 'jot__b'),
 (datetime.date(2021, 2, 14), 'rebelpacifist'),
 (datetime.date(2021, 2, 18), 'neetuanjle_nitu'),
 (datetime.date(2021, 2, 15), 'jot__b'),
 (datetime.date(2021, 2, 20), 'MangalJ23056160'),
 (datetime.date(2021, 2, 23), 'Surrypuria'),
 (datetime.date(2021, 2, 19), 'Preetm91')]

#### Q1 TIME



Se condira valores por defecto como batch_size = 1024, y low_memory=True dado que necesitamos también no utilizar mucha memoria. Se llego a la respuesta final utilizando operaciones de la API de LazyDataFrames para hacer uso del mayor beneficio de vectorización y paralelismo.

In [3]:
%%time
from q1_time import q1_time
result = q1_time(output_file_path)

CPU times: user 5.16 s, sys: 789 ms, total: 5.95 s
Wall time: 5.91 s


In [5]:
from q1_time import q1_time
%timeit result = q1_time(output_file_path)

5.69 s ± 285 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Resultados:

In [6]:
result

[(datetime.date(2021, 2, 12), 'RanbirS00614606'),
 (datetime.date(2021, 2, 13), 'MaanDee08215437'),
 (datetime.date(2021, 2, 17), 'RaaJVinderkaur'),
 (datetime.date(2021, 2, 16), 'jot__b'),
 (datetime.date(2021, 2, 14), 'rebelpacifist'),
 (datetime.date(2021, 2, 18), 'neetuanjle_nitu'),
 (datetime.date(2021, 2, 20), 'MangalJ23056160'),
 (datetime.date(2021, 2, 15), 'jot__b'),
 (datetime.date(2021, 2, 19), 'Preetm91'),
 (datetime.date(2021, 2, 23), 'Surrypuria')]

### Q2 
Los top 10 emojis más usados con su respectivo conteo. 



Para resolver esta pregunta se ha utilizado la librería [Emoji](https://carpedm20.github.io/emoji/docs/).
Hubieron otras alternativas cómo hacer web scrapping de la página dónde estan almacenados los caracteres [unicode.org](https://www.unicode.org/reports/tr51/#emoji_data); sin embargo, para opciones prácticas y no reinventar la rueda se procedió a utilizar la librería Emoji. Además en la búsqueda de los emojis se consideró por un momento utilizar expresiones regulares, pero podían aparecer errores como esta detallado [aquí](https://carpedm20.github.io/emoji/docs/#regular-expression). Por lo tanto, se utilizo la función emoji_list.

#### Q2 Memory

Todos los campos a excepción del "date","user" fueron cambiados de tipo al bool, para ocupar menos espacio en memoria.


In [1]:
from q2_memory import q2_memory
result = q2_memory(output_file_path)

Filename: /home/admintdp/Documentos/personalProyects/challenge_DE/src/q2_memory.py

Line #    Mem usage    Increment  Occurrences   Line Contents
     9 145.3945 MiB 145.3945 MiB           1   @profile(precision=4)
    10                                         def q2_memory(file_path: str) -> List[Tuple[str, int]]:
    11 145.5195 MiB   0.1250 MiB           2       tweets = pd.read_json(
    12 145.3945 MiB   0.0000 MiB           1           file_path,
    13 145.3945 MiB   0.0000 MiB           1           lines=True,
    14 145.3945 MiB   0.0000 MiB           1           chunksize=1000,
    15 145.3945 MiB   0.0000 MiB           1           convert_dates=True,
    16 145.3945 MiB   0.0000 MiB          19           dtype={
    17 145.3945 MiB   0.0000 MiB           1               "url": "bool",
    18 145.3945 MiB   0.0000 MiB           1               "date": "bool",
    19 145.3945 MiB   0.0000 MiB           1               "content": "string",
    20 145.3945 MiB   0.0000 MiB     

Resultados:

In [3]:
result

[('🙏', 5049),
 ('😂', 3072),
 ('🚜', 2972),
 ('🌾', 2182),
 ('🇮🇳', 2086),
 ('🤣', 1668),
 ('✊', 1651),
 ('❤️', 1382),
 ('🙏🏻', 1317),
 ('💚', 1040)]

#### Q2 Time

Se utilizó la librería Polaras para el proceso de extracción de datos, sin embargo, para la respuesta final se hizo uso
de una libreria Collections y la estructura Counter.

In [5]:
%%time
from q2_time import q2_time
output_file_path = "datasets/farmers-protest-tweets-2021-2-4.json"
result = q2_time(output_file_path)

CPU times: user 25.8 s, sys: 160 ms, total: 26 s
Wall time: 26 s


Resultados:

In [2]:
result

[('🙏', 5049),
 ('😂', 3072),
 ('🚜', 2972),
 ('🌾', 2182),
 ('🇮🇳', 2086),
 ('🤣', 1668),
 ('✊', 1651),
 ('❤️', 1382),
 ('🙏🏻', 1317),
 ('💚', 1040)]

### Q3
El top 10 histórico de usuarios (username) más influyentes en función del conteo de las menciones (@) que registra cada uno de ellos.

#### Q3_memory

Todos los campos a excepción del "mentionedUser" fueron cambiados de tipo al bool, para ocupar menos espacio en memoria.
Además se manejó el caso de mentionedUser nulos.

In [1]:
from q3_memory import q3_memory
result = q3_memory(output_file_path)

Filename: /home/admintdp/Documentos/personalProyects/challenge_DE/src/q3_memory.py

Line #    Mem usage    Increment  Occurrences   Line Contents
     7 137.2617 MiB 137.2617 MiB           1   @profile(precision=4)
     8                                         def q3_memory(file_path: str) -> List[Tuple[str, int]]:
     9 137.2617 MiB   0.0000 MiB           2       tweets = pd.read_json(
    10 137.2617 MiB   0.0000 MiB           1           file_path,
    11 137.2617 MiB   0.0000 MiB           1           lines=True,
    12 137.2617 MiB   0.0000 MiB           1           chunksize=1000,
    13 137.2617 MiB   0.0000 MiB           1           convert_dates=True,
    14 137.2617 MiB   0.0000 MiB          19           dtype={
    15 137.2617 MiB   0.0000 MiB           1               "url": "bool",
    16 137.2617 MiB   0.0000 MiB           1               "date": "bool",
    17 137.2617 MiB   0.0000 MiB           1               "content": "bool",
    18 137.2617 MiB   0.0000 MiB       

Resultados:

In [2]:
result

[('narendramodi', 2265),
 ('Kisanektamorcha', 1840),
 ('RakeshTikaitBKU', 1644),
 ('PMOIndia', 1427),
 ('RahulGandhi', 1146),
 ('GretaThunberg', 1048),
 ('RaviSinghKA', 1019),
 ('rihanna', 986),
 ('UNHumanRights', 962),
 ('meenaharris', 926)]

#### Q3_time

Se tuvo que considerar el caso de los mentionedUsers nulos al momento de obtener la lista de usuarios por cada tweet.
Se utilizó la librería Polaras para el proceso de extracción de datos, sin embargo, para la respuesta final se hizo uso
de una libreria Collections y la estructura Counter.

In [2]:
%%time
from q3_time import q3_time_polars
result = q3_time_polars(output_file_path)

CPU times: user 4.63 s, sys: 728 ms, total: 5.36 s
Wall time: 5.38 s


Resultado:

In [2]:
result

[('narendramodi', 2265),
 ('Kisanektamorcha', 1840),
 ('RakeshTikaitBKU', 1644),
 ('PMOIndia', 1427),
 ('RahulGandhi', 1146),
 ('GretaThunberg', 1048),
 ('RaviSinghKA', 1019),
 ('rihanna', 986),
 ('UNHumanRights', 962),
 ('meenaharris', 926)]